In [ ]:
!conda install -c conda-forge jupyter_nbextensions_configurator -y

# Revenue response regression boosting algorithm for multiple treatments

Supervisor: Robin M. Gubela

Ströbel, Jessica 
<br>
Ma, Shih-Chi

## 1. Introduction

Predictive models support decision-making by exploiting the underlying patterns in the data to extract insights for future use. In the real world, we often need to select one condition that gives us the "best" outcome. Examples could be treatment choice: for a certain disease, the doctors need to decide which medicine will be the most effective for the patients; or if we consider direct marketing campaigns where the customers receive emails for advertisements, the company would  like to send the customer the message that yields the highest revenue. Take the latter example, traditionally a classification model selects those customers who should be targeted. That is, the customers having the highest probabilities of  purchasing after receiving the email will be chosen.
<p>
    However, this is not what exactly desired -- there are customers who buy products even without receiving any emails, and targeting them will not increase our revenue, instead, we lose the cost of conducting the marketing campaign on them. Also, there exist the chance that the customers get annoyed by the campaign and don't want to buy any products anymore even though they would without getting the promotion emails. Hence, we want to know the causal relationship between the campaign and the purchasing behaviour of the customers. And this is where uplift modeling comes into play.
<p>
    Uplift models were mainly used for conversion setting where the outcome variable is binary, for instance, the model tells us if a customer will buy a product or not, but there are not so many of them deal with continuous outcome, that is, instead of only predicting yes or no, the model gives prediction on how much money the customer is going to spend due to the campaign, or say, treatment. Except from the target variable, we also want to cope with multiple treatment cases instead of single treatment. Consider the email marketing example again, we now have different versions of email that are going to be sent to the potential target customers, in this case, we want to not only know if the email would increase the sales or not, but also which email would do so the best.
<p>
    On that account, the goal of this paper is to develop an uplift model that handles the case of both multiple treatments and continuous outcome. Our model has its foundation based on the Contextual Treatment Selection (CTS) developed by Zhao 2017 <font color=green>[13]</font>,  combined with AdaBoost algorithms. Boosting often dramatically improves performance of classification models, so we make use of this property and aim to improve the uplift model performance under multiple treatments and continuous outcome.

## 2. Literature Review

Uplift modeling is an approach used for estimating an effect of a treatment on the individual level. It models the incremental effect of a treatment on the target group and should help targeting only those people, who buy because they are subject to a treatment and wouldn’t buy otherwise. With the goal of improving user experience and engagement, it is often widely used in the fields of direct marketing and product offering (Zhao and Harinen 2019) <font color=green>[14]</font>. Other fields of application are for instance personalized medicine to find out for whom the medical treatment works best (Jaskowski and Jaroszewicz 2012 <font color=green>[7]</font>) or the banking or telecommunication sector (Radcliffe and Surry 2011 <font color=green>[10]</font>). Uplift models can be differentiated into two different kinds of models: conversion and revenue uplift models. In contrast to conversion models which aim at maximizing incremental sales, revenue uplift models strive at maximizing incremental revenue. This is especially reasonable when customers differ in their spending <font color=green>[3]</font>.
<p>
    A further distinction of uplift models can be made regarding the number of used treatments, for instance the number of different coupons values provided to the customers in a marketing campaign. While many uplift models covered in literature  discuss only single treatment, the focus is shifting more and more towards models considering multiple treatments. One reason herefore is the rather rare consideration of a single treatment in a real world business setting. Often companies want to compare different forms of promotions, for instance different channels or individual discount codes. Despite its relevance in practical applications, literature on multitreatment uplift is rather limited.
<p>
    The most basic form of measuring uplift is the Separate Model Approach (SMA), where two separate models are built independently from each other on the treatment and the control data.The SMA suggests to calculate the uplift by simply subtracting the estimated probability of the two separate models. Due to its simplicity and separate models for the treatment and control group it is possible to extend it to multiple treatments, whereby a separate model is build for each treatment (Zhao et al. 2017 <font color=green>[13]</font>). A clear advantage of the Two Model Approach is its simplicity and intuitive clarity, which is why it is often used to calculate benchmarks. There is also no need to develop new algorithms since it works with any state-of-the-art machine learning algorithm like XGBoost (Chen and Guestrin 2016 <font color=green>[1]</font>). However, the performance suffers from focusing on building two separate models. As a consequence, the models cannot pick up arelatively weak uplift signal compared to purchase proportion in control and treatment group(Radcliffe and Surry 2011 <font color=green>[10]</font>) and is often outperformed by other uplift approaches (Lo 2002 <font color=green>[8]</font>).
<p>
    Besides the Two Model Approach, another approach to model uplift is to model uplift directly through adapting existing machine learning algorithms like decision trees. A single model isused to predict uplift directly by considering both treatment and control groups. (As a result, it allows optimizing directly the uplift estimation.) In 2012, Rzepakowsi and Jaroszewicz (2012)<font color=green>[7]</font> introduced a decision tree uplift model with multiple treatments and customized splitting and pruning criteria. The primary objective in their model for splitting a unified uplift decision tree is to in increase the divergence of outcome distribution between the treatments and between the treatment and the control group. To evaluate each possible split, measures of divergence between class distributions like the squared Euclidian distance, chi-squared divergenceand Kullback-Leibler divergence are applied.
<p>
    Guelman et al. (2012) <font color=green>[4]</font> extended this approach by using a Random Forest rather than asingle decision tree. The result of the Random Forest uplift model can be calculated by simply taking the average from each tree.
    <p>
    In addition, Zhao et al. (2017) <font color=green>[13]</font> proposed a new direct uplift algorithm, which is a tree-based ensemble algorithm. The unique Splitting Criterion of the so called Contextual Treatment Se-lection (CTS) algorithm builds the decision trees in a way that it directly maximizes the expected response on the training set. At each step of the tree-growing process, the split that brings the greatest increase in expected response is chosen. What is special about CTS is that it can handle both multiple treatments and continuous outcomes. Uplift models are not only being continuously improved in terms of their applicability regarding multiple treatments, but also becoming more and more sophisticated and accurate in their prediction. One obvious approach for improving the performance of any classifier is the usage of Boosting, which can enhance the performance of a model remarkably. The basic idea of boosting is to incorporate many weak classifiers to obtain a better result in the end. A widespread application of Boosting is the AdaBoost algorithm, which combines three basic ideas: First, many weak classifiers arecombined to build a strong classifier. Second, some of the weak classifiers get to say more than the others in the final model and last but not least, each new simple classifier is built by taking the errors of the previous one into account (Freund and Schapire, 1994 <font color=green>[2]</font>). One successful attempt of adapting AdaBoost to uplift models is made by Soltys and Jaroszewicz (2018) <font color=green>[12]</font>. In their paper, they describe three properties that an uplift boosting algorithm should have. Based on that, they introduce three different uplift boosting algorithms, each satisfying two ofthe three properties. Nonetheless, the model proposed in this paper only deals with single treatment cases.

## 3. Uplift AdaBoost

With the goal of building a model that deals with both “multiple treatment” and “continuoustarget variable”, we combined the CTS model from Zhao et al. (2017) [13] and AdaBoostalgorithm, in pursuit of a better prediction. In order to incorporate boosting algorithms intouplift modeling, we found the paper from Sołtys and Jaroszewicz [12] especially helpful. Theproposed model in this paper combined adaboost and single-treatment uplift, and inspiredby this, we designed an Adaboost uplift model that handles multiple treatment. Hence, thealgorithm proposed by us aims at predicting the best treatment for each observation and givingthe predicted response under this predicted treatment.Before introducing the algorithm, we first describe the notation used throughout this paper.

## 3.1 Notation

In this paper, upper case letters are used to denote random variables and lower case letters the realizations; boldface characters stand for vectors and normal typeface for scalars.
1. <b>$X$ </b> is the feature vector andxis its realization. We used Superscripts to indicate specificfeatures. For example, $X_j$ is the $j$ th feature in the vector and $x_j$ its realization.
2. <b>$T$</b> represents the treatment. Assuming𝐾different treatments encoded as ${1, . . . , K}$. Thecontrol group is indicated by $T= 0$.
3. <b>$Y$</b> denotes the response and𝑦the realization. Throughout this paper we assume thelarger the value of $Y$, the more desirable the outcome.

## 3.2 Uplift analogue for Prediction Error

AdaBoost uses an iterative approach to learn from the mistakes of the previous classifiers, so in order to improve the model performance with AdaBoost algorithm, we need a definition of the prediction error.
However, unlike standard classification, due to the innate restriction of uplift models --there is no true answer for the prediction since an observation can never be treated with different treatments at the same time, we are not able to clearly identify whether an uplift model correctly classified a given observation to a treatment or not. To deal with this issue, we defined an approximate proxy of classification error for our uplift model. 

<img src="./paper_image/Pseudo_code.png" style="width: 800px;"/>

where $t$ is the predicted best treatment,  $t^{(i)}$ the actual treatment, $\hat{y}$ the predicted response, and $y_i$ the actual response.
<p>
The intuition behind this proxy is rather simple: if an observation belongs to a treatment group and the model predicts that it should receive another treatment $(t^{(i)} \neq t)$, then the model is only correct if the expected response from the recommended treatment is higher than the original treatment $(\hat{y} >  y_i)$, otherwise, the prediction is wrong.
    <p>
On the other hand, for the case that the model suggests the same treatment as the observation was actually assigned, based on CTS mechanism, we know that the treatment prediction is based on the average response within a child node, that is, if an observation has an predicted response larger than its actual response, that means the observation might get this treatment recommendation just because if fell into a node that has a higher average response from other "high-response observations", and this happens when misclassification happens, the observation does not belong to this specific treatment group, on the contrary, if the actual response is higher than the predicted response even when the recommended treatment is the same as the actual treatment, it means that the observation has just high actual response, and since the CTS selects the maximal average response already, there will be no other classifications that lead to even higher expected response. Hence, in this case, we see it as a correct classification.

## 3.3 Pseudo Code

<img src="./paper_image/Pseudo_code.png" style="width: 800px;"/>

The algorithm has a really similar general-structure to the original Adaboost: after the first predictions are derived, a second model is devised to correct the error from the first model. By decreasing the weights of the wrongly predicted observations, each weak learner focuses on the error of the previous one. Therefore, we will assign weights to each data points, and readjust the weights after each iteration. On top of that, the weak learners are also given corresponding weights to their prediction power for the later-on aggregation in order to obtain final predictions.
<p>
From the pseudo code above, we can see that after assigning initial weights to the observations, we first fit a CTS model $h_m$ to the dataset $D^{t}$ and get the prediction for both treatment and expected response, then according to the prediction error defined before, we can obtain the model error rate --the proportion of wrongly predicted data points with regard to data weights $w_i$ from the formula <font color=blue>$1$</font> given in step  <font color=blue>$2b$</font>.
<p>
With the model error rate, we can compute the model weight. When aggregating all the models from each iterations to derive the final results, we want to have higher contribution from those models with stronger predictive power, and less impact from the weak ones.Thus, we design the model weight formula in <font color=blue>$2c$</font> in the way that the model error rate having a negative relationship with the model weight --the more wrong predictions a model has, the higher the error rate, and as a consequence, the smaller the model weight. Figure <font color=blue>$1$</font> is the graph for formula <font color=blue>$2c$</font>. Since the error rate $\epsilon_m$ is the proportion, it will always be in between $0$ and $1$ as marked in blue, it can be seen that the model weight $\beta_m$ will accordingly be also always between $0$ and $1$.

<img src="./paper_image/model_weight.png" style="width: 800px;"/>

The last step before storing the results and going into the next iteration is then update the observation weights. Here, in order to have our algorithm focusing on the wrong predictions, we decrease the observations' weights when they are correctly predicted, on the other hand, we increase the weights for those observations wrongly predicted. Since the model weight  $\beta_m$ is between $0$ and $1$, the observation weights get smaller when multiplied with it; and larger when multiplied with the reciprocal, here we have the observation weights multiplied with not only the reciprocal of the model weights $\beta_m$, but the reciprocal of  $(1-\frac{1}{2}\beta_m)$ for a reason, and this is explained shortly. In addition to decrease the weight for correct predictions and increase the weight for wrong predictions, we strive to also retain one property from the original AdaBoost --when a stronger model, which means it has lower error rate, makes a wrong prediction, we increase the weight of that data point even more; correspondingly, when a weak model gives a correct prediction, we also decrease the weight even more. And this is the reason why we designed the update of the observation weight for wrong predictions in the way that it's multiplied with the reciprocal of  $(1-\frac{1}{2}\beta_m)$ instead of the reciprocal of $\beta_m$.
<p>
The desired properties of the update functions are clearly shown in figure <font color=blue>$2$</font>. The orange line $w_r$ in the graph stands for update for wrong predictions and the blue line $w_c$ for the correct ones. From the graph we can see that $w_r$ is always bigger than $1$, resulting that the updated weights to grow, and the higher the $\beta_m$, the stronger the learner, the faster the $w_r$ grows; on the contrary, $w_c$ is always between $0$ and $1$, which reduces the weights. Furthermore, since it's equal to $\beta_m$, it has the innate characteristics that when $\beta_m$ declines, it falls as well. This is consistent to our wish that the weights drops even more when even a weak model gives correct predictions.

<img src="./paper_image/update_weight.png" style="width: 800px;"/>

## 4. Evaluation Method

As in any predictive model, measuring the performance of an uplift model is essential. However, there is a big difference in the assessment of the performance. While it is possible to compare the actual and predicted outcome of each individual in a predictive model, it is impossible to do the same for an uplift model. An individual cannot receive both a treatment and no treatment at the same time (Radcliffe, 2007) <font color=green>[9]</font>, so we will never know the ground truth-optimal treatment for each individual. This problem of causal inference (Holland,  1968 <font color=green>[6]</font>) leads to the fact that other ways of evaluation are need for the uplift setting.

## 4.1 Uplift Curves

Common measures to evaluate uplift are uplift curves and qini curves (Radcliffe 2007 <font color=green>[9]</font>; Rzepakowski and Jaroszewicz, 2010 <font color=green>[11]</font>}). Both of them were initially introduced for single treatments and need to be adjusted for the multiple treatments as in our case.
<p>
    For the evaluation of our model we will use the uplift curve, which is described in Gutierrez et al. (2017) <font color=green>[5]</font> as: 

$g(t) = (\frac{Y_t^{(T)}}{N_t^{(T)}} - (\frac{Y_t^{(C)}}{N_t^{(C)}}) * (N_t)^{(T)} + N_t^{(C)})$

where $Y^T$ ($Y^C$) depict the sum of the treated (control) individual predicted outcomes and $N^T$ ($N^C$) the number of the treatment (control) group members per bin. 
The idea behind the formula is to rank the individuals in the treatment and control group according to their predictions and then compute the average prediction per decile. 
<br>
To use the formula for multiple treatments, we combine all treated individuals into one big group, independent of their assigned treatment. As a result, only one treatment and one control group remain. 

## 4.2 Expected Outcome

A second approach which can be used to evaluate uplift curves in a multitreatment setting is the so called modified uplift curve. The modified uplift curve was introduced by Zhao et al. (2017) <font color=green>[13]</font> and is based on an expected outcome metric. In a first step, a new random variable Z is defined:

$Z = \sum_{t=o}^{K}\frac{1}{p_t}Y{1}[h(X)=t]{1}[T=t]$

Where $p_t$ is the probability that the assigned and the predicted treatment are the same and $h(X)$ the predicted treatment by the uplift model. Based on a proof using the law of total expectation, this results in the following sample average:

$\bar{z} = \frac{1}{N}\sum_{i=1}^{N}z^{(i)}$

To put it in simple words, only the outcome of those subjects for whom the assigned and the predicted treatment match are used for the evaluation. In order to take different group sizes into account, the group results are weighted. Using the expected response, a modified uplift curve can be drawn. As already known from the uplift curve, the participants are ranked based on their predicted uplift score in descending order and the sample average $\bar{z}$ defined above is computed.  In the resulting graph, the horizontal axis represents the percentage of the treated population according to the recommendation by the model and the vertical axis depicts the expected response. 

## 5. Results

## 5.1 Our Dataset

For the evaluation of the proposed AdaBoost uplift algorithm, data from a campaign of an online shoe retailer is used. The campaign ran for half a year and involves several campaign types. Each consumer visiting the website of the shoe retailer was randomly allocated to either one of 4 treatment groups or the control group. In the control group, the consumers receive no communications. In the treatment groups, the assigned users get either a 20%, 25%, 50% or 75% discount on their next purchase. The goal of the campaign is to motivate the website visitors to buy something on the website. In the campaign, there isn’t a minimum order value. Discounts can be used for every purchase no matter how low or high in value.
<p>
The data set consists of 133,595 observations and 21 variables. Each observation describes an individual session of a customer. The data set contains both numeric variables like the money spend at the sessions and categorical variables like the channels through which the website is visited (search, e-mail, affiliate…). Other variables include information about whether the consumer is already known by the online shoe retailer, which campaign type the consumer was assigned to or if the consumer has an item add to his shopping cart.
    <p>
Since the experimental setting of the data collection process was a Randomized controlled trial, best experimental conditions are given. Since the subjects are randomly assigned to the treatment or control groups, all known and unknown factors that might have an influence on the results, are present in all of the treatment and control groups. Therefore, differences in outcome can be considered as effects of the treatments. To get an overview over the treatment group assignment and the initial uplift of the data, table X provides some interesting statistics.

<img src="./paper_image/Table_1_Descripitve_uplift_statistics_of_experimental_data.png" style="width: 800px;"/>

## 5.2 Implementation

Before the data set can be used for modeling, some preparations need to be made. After excluding some variables from the data set like the zip code or the last conversion timestamp, we used one hot encoding to represent the categorical variables as binary vectors. One hot encoding is one of the most common used tools for the transformation of categorical values to numeric values since most algorithms require numeric values as an input.
For training and evaluation of the algorithm, we use 80% of the data set to train the model and 20% to evaluate it. To compare the results of our algorithm, we also use the Separate Model Approach (SMA) and the CTS algorithm. The SMA is based on a random forest as a base learner, which parameters are tuned using grid search. We choose 8 iterations for our Uplift AdaBoost algorithm. For comparability, all models are build and evaluated on the same data. 

## 5.3 Result

The performance of the different models is summarized in the uplift curves of Figure <font color=blue>3</font>. A first look shows that the the different models perform clearly better than the random assignment. While the CTS algorithm has the best results, both CTS and SMA are outperforming the CTS Uplift algorithm. The uplift curves of all the models are not steadily increasing, meaning that targeting as many consumers as possible won’t lead to the best result. This is in line with marketing theory since at a certain point at targeting more and more people, also people who would have responded anyway or for whom the action actually has no impact will be targeted, leading to unnecessary contact costs. Another observation is that all curves are decreasing directly in the beginning. One possible explanation for this is that the models are performing quiet bad and are not able to target the right persons in the beginning as first customers. The models should always prioritize only target those customers that responded because of the action and wouldn’t have responded otherwise. 

<img src="Evaluation_new.png">

Against our expectations, the performance of the AdaBoost Uplift algorithm is only slightly better than a random assignment and cannot outperform other algorithms like CTS or even the SMA. Since we are really convinced of the concept, we try to understand possible causes for the bad performance. An indicator for the bad performance can already be found when looking at the predictions of the money spend by each customer. The predicted amount of money spend is increasing in each round steadily, which should not be the case. A possible cause for the bad performance of the Uplift AdaBoost algorithm is possibly the step of random oversampling. Since many of the cases in the data set have a real spending of 0 euro, all these persons get classified as correct by the algorithm in the beginning. As a consequence, these cases have a low assigned weight and will possibly dropping out of the data frame after weighted sampling. These cases with 0 euro spend initially decreased the overall average, which is why after their dropping out the predicted amount of money spend increases. Therefore, weighted sampling doesn’t seem to be the right approach to incorporate weighting in the process.

## 6. Conclusion

## 6.1 Summary

In this seminar paper, our goal was to develop an uplift algorithm for multiple treatments and continuous outcome, that incorporates some kind of boosting to further improve the performance of uplift models. Therefore, we developed an algorithm based on the CTS algorithm from Zhao et al. (2017) <font color=green>[13]</font> and the AdaBoost algorithm. The algorithm aims at predicting the best treatment for each observation and giving the predicted response under this treatment.
<p>
Unfortunately, the evaluation based on a data set from an online shoe retailer provided unexpectedly bad results. We still are convinced by the basic idea of the boosting uplift algorithm for multiple treatments, but some adjustments need to be made. Overall, all of the three used algorithms performed rather poor, leading to the suggestions that there is still much room for improvement.

## 6.2 Limitations and Outlook

Since we believe that the rather poor performance of the Uplift AdaBoost algorithm is caused by the random sampling procedure, it would be interesting to see, if the sampling procedure can be included in another way into the algorithm. One possible idea is to integrate random sampling directly into the CTS algorithm of Zhao et al. (2017) <font color=green>[13]</font>, in the way that when the average response in each child node is calculated, instead of taking the simple average, adopting weighed mean would help to avoid the left-out samples resulting from random sampling with the observation weight.
<p>
For future research it is also interesting to consider the cost factor of the treatment in the Uplift AdaBoost algorithm. When sending discount coupons to customers, the costs for the company should also always be kept in mind. A high discount significantly reduces the company's profit and is therefore should also be considered in the customer targeting decision.

## 7. Reference

[1] Tianqi Chen and Carlos Guestrin. Xgboost: A scalable tree boosting system. In <i>Pro-ceedings of the 22nd acm sigkdd international conference on knowledge discovery and datamining</i>, pages 785–794, 2016.
<br>
[2] Yoav Freund and Robert E Schapire. A decision-theoretic generalization of on-line learningand an application to boosting. <i>Journal of computer and system sciences</i>, 55(1):119–139,1997.
<br>
[3] Robin Marco Gubela, Stefan Lessmann, Johannes Haupt, Annika Baumann, TillmannRadmer, and Fabian Gebert. Revenue uplift modeling. In <i>ICIS</i>, 2017.
<br>
[4] Leo Guelman, Montserrat Guillén, and Ana M Pérez-Marín. Random forests for upliftmodeling: an insurance customer retention case. In <i>International Conference on Modelingand Simulation in Engineering, Economics and Management</i>, pages 123–133. Springer,2012.
<br>
[5] Pierre Gutierrez and Jean-Yves Gérardy. Causal inference and uplift modelling: A reviewof the literature. In <i>International Conference on Predictive Applications and APIs</i>, pages1–13, 2017.
<br>
[6] Paul W Holland. Statistics and causal inference. <i>Journal of the American statisticalAssociation</i>, 81(396):945–960, 1986.
<br>
[7] Maciej Jaskowski and Szymon Jaroszewicz. Uplift modeling for clinical trial data. In <i>ICMLWorkshop on Clinical Data Analysis</i>, 2012.
<br>
[8] Chung-Mau Lo, Henry Ngan, Wai-Kuen Tso, Chi-Leung Liu, Chi-Ming Lam, Ronnie Tung-Ping Poon, Sheung-Tat Fan, and John Wong. Randomized controlled trial of transarte-rial lipiodol chemoembolization for unresectable hepatocellular carcinoma. <i>Hepatology</i>, 35(5):1164–1171, 2002.
<br>
[9] Nicholas J Radcliffe. Using control groups to target on predicted lift: Building and assessinguplift models. <i>Direct Marketing Analytics Journal</i>, 1(3):14–21, 2007.
<br>
[10] Nicholas J Radcliffe and Patrick D Surry. Real-world uplift modelling with significance-based uplift trees. <i>White Paper TR-2011-1, Stochastic Solutions</i>, pages 1–33, 2011.
<br>
[11] Piotr Rzepakowski and Szymon Jaroszewicz. Decision trees for uplift modeling. In <i>2010IEEE International Conference on Data Mining</i>, pages 441–450. IEEE, 2010.
<br>
[12] Michał Sołtys and Szymon Jaroszewicz. Boosting algorithms for uplift modeling. <i>arXivpreprint arXiv:1807.07909</i>, 2018.
<br>
[13]Yan Zhao, Xiao Fang, and David Simchi-Levi. <i>Uplift Modeling with Multiple Treatmentsand General Response Types</i>, pages 588–596.
<br>
[14] Zhenyu Zhao and Totte Harinen. Uplift modeling for multiple treatments with cost opti-mization. In <i>2019 IEEE International Conference on Data Science and Advanced Analytics(DSAA)</i>, pages 422–431. IEEE, 2019.